In [1]:
import pandas as pd
import numpy as np
import random
import os
import torch
import sys
sys.path.append('../')
from lillian_modules import utils, simple_constants
from lillian_modules.simple_env import SimpleEnv
from stable_baselines3.dqn import DQN
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler
import warnings
warnings.filterwarnings('ignore')

In [2]:
SEED = simple_constants.SEED
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [3]:
class_dict = simple_constants.CLASS_DICT

In [4]:
# frac=0.5

#### The data

In [5]:
# def balance_dataframe(df):
#     target_num = df.label.value_counts().min()
#     sample_amounts = {0: target_num, 1:target_num}

#     s = (df.groupby('label').apply(lambda g: g.sample(n=sample_amounts[g.name], 
#                                                       replace=len(g) < sample_amounts[g.name])
#                                   ).reset_index(drop=True))
#     return s

In [6]:
# df = pd.read_csv('../../RL-Agent-Diagnosis/lupus/data/orig/lupus_dataset.csv')
# df = pd.read_csv('../../RL-Agent-Diagnosis/lupus/data/very_simple_datasets/feats_8.csv')
# df.head()

In [7]:
# df.label.value_counts()

In [8]:
# df['label'] = df['label'].replace(class_dict)
# # print(df.label.value_counts())
# X = df.iloc[:, 0:-1]
# y = df.iloc[:, -1]

# X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=SEED)
# X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.10, stratify=y_train_val, random_state=SEED)
# training_df = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
# validation_df = pd.concat([X_val, y_val], axis=1).reset_index(drop=True)
# testing_df = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)
training_df = pd.read_csv('../../RL-Agent-Diagnosis/lupus/data/missingness/0/training_set.csv')
validation_df = pd.read_csv('../../RL-Agent-Diagnosis/lupus/data/missingness/0/validation_set.csv')
testing_df = pd.read_csv('../../RL-Agent-Diagnosis/lupus/data/missingness/0/testing_set.csv')
# training_df = pd.read_csv(f'../../RL-Agent-Diagnosis/lupus/data/noisiness/{frac}/training_set.csv')
# validation_df = pd.read_csv(f'../../RL-Agent-Diagnosis/lupus/data/noisiness/{frac}/validation_set.csv')
# testing_df = pd.read_csv(f'../../RL-Agent-Diagnosis/lupus/data/noisiness/{frac}/testing_set.csv')
training_df = training_df.fillna(-1)
X_train, y_train = training_df.iloc[:, 0:-1], training_df.iloc[:, -1]
X_val, y_val = validation_df.iloc[:, 0:-1], validation_df.iloc[:, -1]
X_test, y_test = testing_df.iloc[:, 0:-1], testing_df.iloc[:, -1]
# y = df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((50400, 23), (5600, 23), (14000, 23), (50400,), (5600,), (14000,))

In [9]:
testing_df.isna().sum()

ana                             0
fever                           0
leukopenia                      0
thrombocytopenia                0
auto_immune_hemolysis           0
delirium                        0
psychosis                       0
seizure                         0
non_scarring_alopecia           0
oral_ulcers                     0
cutaneous_lupus                 0
pleural_effusion                0
pericardial_effusion            0
acute_pericarditis              0
joint_involvement               0
proteinuria                     0
anti_cardioliphin_antibodies    0
anti_β2gp1_antibodies           0
lupus_anti_coagulant            0
low_c3                          0
low_c4                          0
anti_dsdna_antibody             0
anti_smith_antibody             0
label                           0
dtype: int64

In [10]:
unique, counts = np.unique(y_train, return_counts=True)
unique, counts

(array([0, 1]), array([26203, 24197]))

In [11]:
# len(balanced_training_df[(balanced_training_df.ana==1) & (balanced_training_df.label==0)])

In [12]:
len(validation_df[(validation_df.ana==1) &(validation_df.label==0)])

1265

In [13]:
# len(balanced_training_df)

#### Training the model

In [14]:
def stable_dqn3(X_train, y_train, timesteps, validation_times=10, folder_name=None):
    training_env = SimpleEnv(X_train, y_train)
    print(f'ACTION SPACE: {training_env.actions}')
    model = DQN(policy='MlpPolicy', env=training_env, verbose=1, seed=SEED, X_val=X_val, y_val=y_val, 
                to_save_folder=folder_name)
    model.learn(total_timesteps=timesteps, validation_times=validation_times, log_interval=100000)
#     if save:
#         model.save(filename)
    training_env.close()
    return model

In [ ]:
ft_num = 23
steps = 70000000

# for steps in [int(6e6)]:
# for steps in [int()]:
model = stable_dqn3(X_train, y_train, timesteps=steps, validation_times=700,
                    folder_name=f'../models/very_simple_models/negative_step_reward/unbalanced/14_03_23/')
# val_df.to_csv(f'../test_dfs/val_df_ft_{ft_num}_{steps}', index=False)
#     stable_dqn3(X_train, y_train, steps)

ACTION SPACE: ['No lupus', 'Lupus', 'Inconclusive diagnosis', 'ana', 'fever', 'leukopenia', 'thrombocytopenia', 'auto_immune_hemolysis', 'delirium', 'psychosis', 'seizure', 'non_scarring_alopecia', 'oral_ulcers', 'cutaneous_lupus', 'pleural_effusion', 'pericardial_effusion', 'acute_pericarditis', 'joint_involvement', 'proteinuria', 'anti_cardioliphin_antibodies', 'anti_β2gp1_antibodies', 'lupus_anti_coagulant', 'low_c3', 'low_c4', 'anti_dsdna_antibody', 'anti_smith_antibody']
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
setting up model
Validating at 100000/70000000
Validating at 200000/70000000
Validating at 300000/70000000
Validating at 400000/70000000
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.64     |
|    ep_rew_mean      | -3.74    |
|    exploration_rate | 0.936    |
|    success_rate     | 0.21     |
| time/               |          |
|    episodes         | 100000   |
|    fps

Validating at 7200000/70000000
Validating at 7300000/70000000
Validating at 7400000/70000000
Validating at 7500000/70000000
Validating at 7600000/70000000
Validating at 7700000/70000000
Validating at 7800000/70000000
Validating at 7900000/70000000
Validating at 8000000/70000000
Validating at 8100000/70000000
Validating at 8200000/70000000
Validating at 8300000/70000000
Validating at 8400000/70000000
Validating at 8500000/70000000
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 14.4     |
|    ep_rew_mean      | -2.03    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.57     |
| time/               |          |
|    episodes         | 1100000  |
|    fps              | 2085     |
|    time_elapsed     | 4107     |
|    total_timesteps  | 8564977  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0187   |
|    n_updates        | 2128744  |
----------------------------------
Valida

Validating at 18900000/70000000
Validating at 19000000/70000000
Validating at 19100000/70000000
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 11.6     |
|    ep_rew_mean      | -1       |
|    exploration_rate | 0.05     |
|    success_rate     | 0.73     |
| time/               |          |
|    episodes         | 1900000  |
|    fps              | 2060     |
|    time_elapsed     | 9317     |
|    total_timesteps  | 19196025 |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0717   |
|    n_updates        | 4786506  |
----------------------------------
Validating at 19200000/70000000
Validating at 19300000/70000000
Validating at 19400000/70000000
Validating at 19500000/70000000
Validating at 19600000/70000000
Validating at 19700000/70000000
Validating at 19800000/70000000
Validating at 19900000/70000000
Validating at 20000000/70000000
Validating at 20100000/70000000
Validating at 20200000/7

In [ ]:
# model = utils.load_dqn3('../models/very_simple_models/negative_step_reward/unbalanced/23_ft_incorr_0_corr_100/best_roc_auc_model.zip')

In [ ]:
# timesteps = 1000000
# ft_num = 8
# training_env = SimpleEnv(X_train, y_train)
# # training_env = LupusEnv(X_train, y_train)
# # model = DQN('MlpPolicy', training_env, verbose=1, seed=SEED, learning_rate=0.00001)
# model = DQN('MlpPolicy', training_env, verbose=1, seed=simple_constants.SEED)
# model.learn(total_timesteps=timesteps, log_interval=100000)
# # model.save(f'../models/very_simple_models/reward_shaping/dqn_fts_{ft_num}_{timesteps}')
# training_env.close()

#### Testing the model

In [ ]:
# test_df = pd.DataFrame()
# testing_env = SimpleEnv(X_test, y_test, random=False)
# # testing_env = LupusEnv(X_test, y_test, random=False)
# count = 0
# try:
#     while True:
#         count += 1
#         if count %(len(X_test)/5)==0:
#             print(f'Count: {count}')
#         obs, done = testing_env.reset(), False
#         while not done:
#             action, states = model.predict(obs, deterministic=True)
#             obs, rew, done, info = testing_env.step(action)
#             if done==True:
#                 test_df = test_df.append(info, ignore_index=True)
# except StopIteration:
#     print('Testing done ....')
# test_df.head() 

## test_df.episode_length.min(), test_df.episode_length.max()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
# test_df.to_csv('../../RL-Agent-Diagnosis/lupus/test_dfs/noisiness/0.5.csv', index=False)

In [ ]:
a = -1
b = np.nan

In [ ]:
if b > a:
    print('yes')
else:
    print('no')

In [ ]:
c = b*100
c